In [1]:
"""
2^n
"""
def cut_rod(p, n):
    if n == 0:
        return 0
        
    q = float('-inf')
    
    for i in range(n):
        q = max(q, p[i] + cut_rod(p, n - 1))
        
    return q

In [4]:
def memoized_cut_rod(p, n):
    r = [float("-inf")] * (n + 1)
    r[0] = 0
    return memoized_cut_rod_aux(p, n, r)

def memoized_cut_rod_aux(p, n, r):
    if r[n] >= 0:
        return r[n]
        
    q = float("-inf")
    for i in range(n):
        q = max(q, p[i] + memoized_cut_rod_aux(p, n - i - 1, r))
        
    r[n] = q
    return q

def bottom_up_cut_rod(p, n):
    r = [0] * (n + 1)
    
    for j in range(1, n + 1):
        q = float("-inf")
        
        for i in range(j):
            q = max(q, p[i] + r[j - i - 1])
        r[j] = q
    
    return r[n]

In [8]:
def extended_bottom_up_cut_rod(p, n):
    r = [0] * (n + 1)
    s = [0] * (n + 1)
    
    for j in range(1, n + 1):
        q = float("-inf")
        for i in range(1, j + 1):
            value = p[i] + r[j - i]
            if q < value:
                q = value
                s[j] = i
        r[j] = q
    return r, s

def print_cut_rod_solution(p, n):
    (r, s) = extended_bottom_up_cut_rod(p, n)
    
    while n > 0:
        print(s[n])
        n = n - s[n]

In [9]:
"""
p = [0,1,5,8,9,10,17,17,20,24,30]
n = 7

optimal solution is:
Revenue = 18
Cuts: 1, 6

The `memoized_cut_rod_aux` starts at 0 to n
`memoized_cut_rod_aux_rev` Starts at n and goes backwards
"""
def memoized_cut_rod_rev(p, n):
    r = [-1] * (n + 1)
    s = [0] * (n + 1)

    revenue = memoized_cut_rod_aux(p, n, r, s)
    return revenue, s


def memoized_cut_rod_aux_rev(p, n, r, s):
    if r[n] >= 0:
        return r[n]

    if n == 0:
        q = 0
    else:
        q = float('-inf')

        for i in range(1, n + 1):
            value = p[i] + memoized_cut_rod_aux(p, n - i, r, s)
            if value > q:
                q = value
                s[n] = i

    r[n] = q
    return q

In [10]:
def fibonacci(n):
    if n <= 1:
        return n

    prev2 = 0
    prev1 = 1

    for _ in range(2, n + 1):
        current = prev1 + prev2
        prev2 = prev1
        prev1 = current

    return prev1

In [12]:
def rectangular_matrix_multiply(A, B, C, p, q, r):
    for i in range(p):
        for j in range(r):
            C[i][j] = 0
            for k in range(q):
                C[i][j] += A[i][k] * B[k][j]

def rectangular_matrix_multiply_cleaner(A, B):
    p = len(A)
    q = len(A[0])
    r = len(B[0])

    C = [[0 for _ in range(r)] for _ in range(p)]

    for i in range(p):
        for j in range(r):
            for k in range(q):
                C[i][j] += A[i][k] * B[k][j]

    return C

In [14]:
"""
A1 = 5×4
A2 = 4×6
A3 = 6×2

p = [5, 4, 6, 2]
m = [[0,0,0],
     [0,0,0],
     [0,0,0]]

i <= j (this forms a triangle matrix)
above the main diagonal

m[0][0] → cost of A1 alone
m[0][1] → cost of A1A2
m[0][2] → cost of A1A2A3

l = chain length
i = start index
j = end index
k = split point

s = i .... k | k+1 .... j
     
"""
def matrix_chain_order(p):
    n = len(p) - 1
    
    m = [[0] * n for _ in range(n)]
    s = [[0] * n for _ in range(n)]
        
    for l in range(2, n + 1):
        for i in range(n - l + 1):
            j = i + l - 1
            m[i][j] = float("inf")
            
            for k in range(i, j):
                q = m[i][k] + m[k + 1][j] + p[i] * p[k + 1] * p[j + 1]
                
                if q < m[i][j]:
                    m[i][j] = q
                    s[i][j] = k
    return m, s

In [16]:
def print_optimal_parens(s, i, j):
    if i == j:
        print(f"A{i+1}", end="")
    else:
        print("(", end="")
        print_optimal_parens(s, i, s[i][j])
        print_optimal_parens(s, s[i][j] + 1, j)
        print(")", end="")

In [17]:
def matrix_chain_multiply_recursive(A, s, i, j):
    if i == j:
        return A[i]
    
    k = s[i][j]
    
    left = matrix_chain_multiply(A, s, i, k)
    right = matrix_chain_multiply(A, s, k + 1, j)
    
    return rectangular_matrix_multiply(left, right)

In [18]:
def recursive_matrix_chain(p, i, j):
    if i == j:
        return 0
        
    min_cost = float("inf")
    
    for k in range(i, j):
        q = (
            recursive_matrix_chain(p, i, k)
            + recursive_matrix_chain(p, k + 1, j)
            + p[i - 1] * p[k] * p[j]
        )
        
        if q < min_cost:
            min_cost = q
            
    return min_cost

In [19]:
def memoized_matrix_chain(p):
    n = len(p) - 1
    m = [[float("inf")] * n for _ in range(n)]
    return lookup_chain(m, p, 0, n - 1)


def lookup_chain(m, p, i, j):
    if m[i][j] < float("inf"):
        return m[i][j]

    if i == j:
        m[i][j] = 0
    else:
        for k in range(i, j):
            q = (
                lookup_chain(m, p, i, k)
                + lookup_chain(m, p, k + 1, j)
                + p[i] * p[k + 1] * p[j + 1]
            )

            if q < m[i][j]:
                m[i][j] = q

    return m[i][j]

A Catalan number is a sequence of natural numbers
that appears in many counting problems involving recursive structures
or balanced arrangements.

$$
C_n = \frac{1}{n+1}\binom{2n}{n}
$$

$$
C_n = \frac{(2n)!}{(n+1)!n!}
$$

$$
C_0 = 1
$$

$$
C_n = \sum_{i=0}^{n-1} C_i C_{n-1-i}
$$

$$
C_n \sim \frac{4^n}{n^{3/2}\sqrt{\pi}}
$$

In [20]:
def lcs_length(x, y):
    m = len(x)
    n = len(y)

    c = [[0] * (n + 1) for _ in range(m + 1)]
    b = [[""] * (n + 1) for _ in range(m + 1)]

    for i in range(1, m + 1):
        for j in range(1, n + 1):
            if x[i - 1] == y[j - 1]:
                c[i][j] = c[i - 1][j - 1] + 1
                b[i][j] = "↖"
            elif c[i - 1][j] >= c[i][j - 1]:
                c[i][j] = c[i - 1][j]
                b[i][j] = "↑"
            else:
                c[i][j] = c[i][j - 1]
                b[i][j] = "←"

    return c, b


def print_lcs(b, x, i, j):
    if i == 0 or j == 0:
        return
    if b[i][j] == "↖":
        print_lcs(b, x, i - 1, j - 1)
        print(x[i - 1], end="")
    elif b[i][j] == "↑":
        print_lcs(b, x, i - 1, j)
    else:
        print_lcs(b, x, i, j - 1)